<h1>C4 Solution</h1>

<h3>Get the data and copy it to S3</h3>

In [11]:
#bucket = 'solutionudacity20210905'
prefix = 'dogImages'

# Define IAM role
import boto3
import re
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role

session = sagemaker.Session()

bucket= session.default_bucket()
print("Default Bucket: {}".format(bucket))

region = session.boto_region_name
print("AWS Region: {}".format(region))

role = get_execution_role()
print("RoleArn: {}".format(role))

Default Bucket: sagemaker-us-east-1-769654117898
AWS Region: us-east-1
RoleArn: arn:aws:iam::769654117898:role/service-role/AmazonSageMaker-ExecutionRole-20211230T131845


In [1]:
%%capture
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip

In [2]:
%%capture
!unzip dogImages.zip

In [10]:
%%capture
!aws s3 cp dogImages s3://sagemaker-us-east-1-769654117898/ --recursive

<h3>Install and import</h3>

In [4]:
%%capture
!pip install smdebug torch torchvision tqdm

In [5]:
import sagemaker
import boto3
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile
import os

<h3>Set up parameters, estimator, and tuner</h3>

In [12]:
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.001, 0.1),
    "batch_size": CategoricalParameter([32, 64, 128, 256, 512]),
}

role = sagemaker.get_execution_role()

objective_metric_name = "Test Loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "Test Loss", "Regex": "Testing Loss: ([0-9\\.]+)"}]

In [13]:
estimator = PyTorch(
    entry_point="hpo.py",
    base_job_name='pytorch_dog_hpo',
    role=role,
    framework_version="1.4.0",
    instance_count=1,
    instance_type="ml.m5.xlarge",
    py_version='py3'
)

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=2,
    max_parallel_jobs=2,
    objective_type=objective_type
)

<h3>Fit the tuner</h3>

In [14]:
os.environ['SM_CHANNEL_TRAINING']=f's3://{bucket}/dogImages'
os.environ['SM_MODEL_DIR']=f's3://{bucket}/dogImages/model/'
os.environ['SM_OUTPUT_DATA_DIR']=f's3://{bucket}/dogImages/output/'
tuner.fit({"training": f's3://{bucket}/dogImages'})

.............................................................................................................................................................................................................................................................................__s

Job ended with status 'Stopped' rather than 'Completed'. This could mean the job timed out or stopped early for some other reason: Consider checking whether it completed as you expect.


<h3>Describe the tuning results</h3>


In [15]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics
latest_tuning_name = tuner.latest_tuning_job.name
print(latest_tuning_name)
exp = HyperparameterTuningJobAnalytics(
  hyperparameter_tuning_job_name=tuner.latest_tuning_job.name)

jobs = exp.dataframe()

jobs.sort_values('FinalObjectiveValue', ascending=0)

pytorch-training-220103-1755


,batch_size,learning_rate,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
1,"""128""",0.006259,pytorch-training-220103-1755-001-52b45cfb,Stopped,180.0,2022-01-03 17:58:10+00:00,2022-01-03 18:18:20+00:00,1210.0
0,"""32""",0.037332,pytorch-training-220103-1755-002-6c33c95e,Completed,150.0,2022-01-03 17:57:57+00:00,2022-01-03 18:16:37+00:00,1120.0


## Imp: If kernel dies, how to continue from a completed training job

In [8]:
#BetterTrainingJobName='pytorch-training-210623-2156-001-fdd5e081'

In [9]:
#my_estimator = sagemaker.estimator.Estimator.attach(BetterTrainingJobName)


In [10]:
#my_estimator.hyperparameters()

In [11]:
#best_estimator=my_estimator

<h3>Prepare to perform Training on Best Estimator</h3>

In [16]:
best_estimator=tuner.best_estimator()


2022-01-03 18:16:37 Starting - Preparing the instances for training
2022-01-03 18:16:37 Downloading - Downloading input data
2022-01-03 18:16:37 Training - Training image download completed. Training in progress.
2022-01-03 18:16:37 Uploading - Uploading generated training model
2022-01-03 18:16:37 Completed - Training job completed


In [17]:
best_estimator.hyperparameters()

{'_tuning_objective_metric': '"Test Loss"',
 'batch_size': '"32"',
 'learning_rate': '0.0373323801869528',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"pytorch_dog_hpo-2022-01-03-17-55-35-402"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-769654117898/pytorch_dog_hpo-2022-01-03-17-55-35-402/source/sourcedir.tar.gz"'}

In [18]:
hyperparameters = {"batch_size": int(best_estimator.hyperparameters()['batch_size'].replace('"', '')), \
                   "learning_rate": best_estimator.hyperparameters()['learning_rate']}
hyperparameters

{'batch_size': 32, 'learning_rate': '0.0373323801869528'}

In [19]:
rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

In [20]:
hook_config = DebuggerHookConfig(
    hook_parameters={
        "train.save_interval": "1",
        "eval.save_interval": "1"
    }
)

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=1)
)

<h2>Creating an Estimator</h2>

In [21]:
#adjust this cell to accomplish multi-instance training
estimator = PyTorch(
    entry_point='hpo.py',
    base_job_name='dog-pytorch',
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    framework_version='1.4.0',
    py_version='py3',
    hyperparameters=hyperparameters,
    ## Debugger and Profiler parameters
    rules = rules,
    debugger_hook_config=hook_config,
    profiler_config=profiler_config,
)

In [22]:
estimator.fit({"training": f's3://{bucket}/dogImages'}, wait=False)

<h2>Creating an Estimator - Multi-Instance Training,</h2>

In [19]:
###in this cell, create and fit an estimator using multi-instance training


<h2>Deployment</h2>

In [26]:
model_location=estimator.model_data


In [27]:
import sagemaker
import boto3
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor


In [28]:
jpeg_serializer = sagemaker.serializers.IdentitySerializer("image/jpeg")
json_deserializer = sagemaker.deserializers.JSONDeserializer()


class ImagePredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImagePredictor, self).__init__(
            endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=jpeg_serializer,
            deserializer=json_deserializer,
        )

In [29]:
pytorch_model = PyTorchModel(model_data=model_location, role=role, entry_point='infernce2.py',py_version='py3',
                             framework_version='1.4',
                             predictor_cls=ImagePredictor)

In [ ]:
predictor = pytorch_model.deploy(initial_instance_count=1, instance_type='ml.m5.large')


--

In [ ]:
import requests
#request_dict={ "url": "https://cdn1-www.cattime.com/assets/uploads/2011/12/file_2744_british-shorthair-460x290-460x290.jpg" }
request_dict={ "url": "https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/20113314/Carolina-Dog-standing-outdoors.jpg" }

img_bytes = requests.get(request_dict['url']).content
type(img_bytes)

In [ ]:
from PIL import Image
import io
Image.open(io.BytesIO(img_bytes))

In [ ]:
response=predictor.predict(img_bytes, initial_args={"ContentType": "image/jpeg"})

In [ ]:
import json
response2=predictor.predict(json.dumps(request_dict), initial_args={"ContentType": "application/json"})

In [ ]:
type(response2[0][0])

In [ ]:
response2[0]

In [ ]:
import torch
import numpy as np
np.argmax(response, 1)